In [1]:
import os
from zipfile import ZipFile
from google.colab import drive

# Drive Mount
drive.mount('/content/drive')

# 알집이 있는 경로에서 open.zip을 가져옵니다
dir = '/content/drive/MyDrive/DACON/Podiblock_Structure_Extraction_AI_Contest/'

# 실제로 알집이 풀어질 경로
base_dir = '/content/'
# os.chdir(base_dir)

# 알집을 풀어볼까용?
ZipFile(dir + 'open.zip').extractall(base_dir)

# 실제로 잘 풀렸는지 확인
os.listdir(base_dir)


Mounted at /content/drive


['.config',
 'test',
 'sample_submission.csv',
 'test.csv',
 'train.csv',
 'drive',
 'train',
 'sample_data']

## Import

In [2]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore') 

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
!nvidia-smi

Wed Jan 11 16:47:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    11W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## wandb

### Setup

In [5]:
!pip install wandb -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.0 MB/s eta 0:00:00


In [6]:
import wandb

# wandb.login()
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
#c5c19470f3b9c096d12e4ff4236c05474afa12b8

### Define the sweep

In [7]:
sweep_config = {
    'method': 'random'
    }

In [8]:
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [9]:
parameters_dict = ({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    })
sweep_config['parameters'] = parameters_dict

In [13]:
parameters_dict.update({
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 127,
        'max': 128,},
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'epochs': {
        'value': 5}
 })

In [14]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 128,
                               'min': 127,
                               'q': 8},
                'epochs': {'value': 5},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


### Initialize the Sweep

In [15]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo3")

Create sweep with ID: briwn70x
Sweep URL: https://wandb.ai/dcv_block-/pytorch-sweeps-demo3/sweeps/briwn70x


### Run your Training procedure

In [16]:
df = pd.read_csv('./train.csv')
df = df.sample(frac=1)
train_len = int(len(df) * 0.8)

train = df[:train_len]
val = df[train_len:]

cv2.imread(train['img_path'].values[0]).shape

train['img_path']

28803    ./train/TRAIN_28803.jpg
23304    ./train/TRAIN_23304.jpg
27036    ./train/TRAIN_27036.jpg
29456    ./train/TRAIN_29456.jpg
16098    ./train/TRAIN_16098.jpg
                  ...           
12126    ./train/TRAIN_12126.jpg
30459    ./train/TRAIN_30459.jpg
18756    ./train/TRAIN_18756.jpg
16804    ./train/TRAIN_16804.jpg
24134    ./train/TRAIN_24134.jpg
Name: img_path, Length: 26395, dtype: object

In [17]:
def get_labels(df):
    return df.iloc[:,2:].values

train_labels = get_labels(train)

In [18]:
print(train_labels)

[[1 0 0 ... 1 1 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 1 ... 1 0 1]
 [1 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]]


In [19]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [20]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=10):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.classifier(x))
        return x

In [21]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_wow(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader = build_dataset(config.batch_size)
        network = build_network()
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch})           

In [22]:
def build_dataset(batch_size):
    
    # wandb.config.batch_size = 128
    img_size = 224
    train_transform = A.Compose([
                                A.Resize(img_size, img_size),
                                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                                ToTensorV2()
                                ])

    train_dataset = CustomDataset(train['img_path'].values, train_labels, train_transform)
    loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, num_workers=0)
    return loader

def build_network():
    network = BaseModel()

    return network.to(device)

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer

def train_epoch(network, loader, optimizer):
    
    criterion = nn.BCELoss().to(device)    
    train_loss = []
    for imgs, labels in tqdm(iter(loader)):
        imgs = imgs.float().to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        output = network(imgs)
        loss = criterion(output, labels)
        
        train_loss.append(loss.item())
        _train_loss= np.mean(train_loss)
        
        loss.backward()
        optimizer.step()
        
        
        wandb.log({"batch loss": loss.item()})
    return _train_loss 

In [ ]:
wandb.agent(sweep_id, train_wow, count=100)

wandb: Agent Starting Run: 8f88mhru with config:
wandb: 	batch_size: 128
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0801180436661108
wandb: 	optimizer: adam
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ugiugi (dcv_block-). Use `wandb login --relogin` to force relogin


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

  0%|          | 0/207 [00:00<?, ?it/s]

  0%|          | 0/207 [00:00<?, ?it/s]

...

  0%|          | 0/207 [00:00<?, ?it/s]

batch loss,▃▃▁▁▂▄▅▆▃▄▄▂▁▄▃▂▆▅▆▆▅▅▄▆▆▄▅▃▅▅█▆▅▅▄▄▅▅▅▃
epoch,▁▃▅▆█
loss,▁████
batch loss,48.88889
epoch,4
loss,51.30464


wandb: Agent Starting Run: ubzsn79z with config:
wandb: 	batch_size: 128
wandb: 	epochs: 5
wandb: 	learning_rate: 0.05139207000770747
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


  0%|          | 0/207 [00:00<?, ?it/s]

  0%|          | 0/207 [00:00<?, ?it/s]

## Hyperparameter Setting

In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':5,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':128,
    'SEED':41
}

## Fixed RandomSeed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Load

In [ ]:
%cd /content

/content


In [ ]:
df = pd.read_csv('./train.csv')

## Train / Validation Split

In [ ]:
df = df.sample(frac=1)
train_len = int(len(df) * 0.8)
print(train_len)

26395


In [ ]:
train = df[:train_len]
val = df[train_len:]

## Data Preprocessing

In [ ]:
def get_labels(df):
    return df.iloc[:,2:].values

In [ ]:
train_labels = get_labels(train)
val_labels = get_labels(val)

## CustomDataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=10):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = F.sigmoid(self.classifier(x))
        return x

# 가속기 설정

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 14.7 MB/s eta 0:00:00


In [ ]:
 from accelerate import Accelerator
 accelerator = Accelerator()


## Train

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_acc = 0
    best_model = None
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            #loss.backward()
            accelerator.backward(loss)

            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_acc)
            
        if best_val_acc < _val_acc:
            best_val_acc = _val_acc
            best_model = model
    
    return best_model

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    val_acc = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.5
            batch_acc = (labels == preds).mean()
            
            val_acc.append(batch_acc)
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_acc = np.mean(val_acc)
    
    return _val_loss, _val_acc

## Run!!

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

0it [00:00, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.29987] Val Loss : [0.15310] Val ACC : [0.93648]


0it [00:00, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

...

  0%|          | 0/52 [00:00<?, ?it/s]

Epoch [4], Train Loss : [0.04133] Val Loss : [0.05692] Val ACC : [0.97793]


0it [00:00, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

Epoch [5], Train Loss : [0.03362] Val Loss : [0.04716] Val ACC : [0.98210]


In [ ]:
 !python --version

Python 3.8.16


## Inference

In [ ]:
test = pd.read_csv('./test.csv')

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += preds.tolist()
    return predictions

In [ ]:
preds = inference(model, test_loader, device)

  0%|          | 0/12 [00:00<?, ?it/s]

## Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit.iloc[:,1:] = preds
submit.head()

,id,A,B,C,D,E,F,G,H,I,J
0,TEST_00000,0,1,1,0,0,1,1,0,1,0
1,TEST_00001,0,0,0,0,1,1,1,0,0,0
2,TEST_00002,1,1,1,1,1,1,0,0,0,1
3,TEST_00003,1,1,1,0,1,1,0,0,1,0
4,TEST_00004,1,1,0,0,1,1,1,0,0,0


In [ ]:
submit.to_csv('/content/drive/MyDrive/DACON/Podiblock_Structure_Extraction_AI_Contest/1_2022.01.06_Mickey_baseline_submit.csv', index=False)